In [34]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GraphConv
from torch_geometric.data import Data

import numpy as np
from utils import Normalizer, set_seed
from conditional_Action_DiT import Conditional_ODE
import matplotlib.pyplot as plt
from discrete import *
import sys
import pdb
import csv

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
n_gradient_steps = 100_000
batch_size = 64
model_size = {"d_model": 256, "n_heads": 4, "depth": 3}
H = 10 # horizon, length of each trajectory

# Define initial and final points, and a single central obstacle
initial_point_up = np.array([0.0, 0.0])
final_point_up = np.array([20.0, 0.0])
final_point_down = np.array([0.0, 0.0])
initial_point_down = np.array([20.0, 0.0])
obstacle = (10, 0, 4.0) 

# Loading training trajectories
all_points1 = []    # want modes 1, 2, 4, 6
all_points2 = []    # want modes 1, 2, 3, 5
with open('data/trajs_noise1.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        x1, y1 = float(row[4]), float(row[5])
        x2, y2 = float(row[7]), float(row[8])
        all_points1.append([x1, y1])
        all_points2.append([x2, y2])

num_trajectories = 10000
points_per_trajectory = 10

expert_data1 = [
    all_points1[i * points_per_trajectory:(i + 1) * points_per_trajectory]
    for i in range(num_trajectories)
]
first_trajectory1 = expert_data1[0]
x1 = [point[0] for point in first_trajectory1]
y1 = [point[1] for point in first_trajectory1]

expert_data2 = [
    all_points2[i * points_per_trajectory:(i + 1) * points_per_trajectory]
    for i in range(num_trajectories)
]
first_trajectory2 = expert_data2[0]
x2 = [point[0] for point in first_trajectory2]
y2 = [point[1] for point in first_trajectory2]


expert_data1 = np.array(expert_data1)
expert_data2 = np.array(expert_data2)


# Unspliced trajectories to get final positions
orig1 = [
    all_points1[i * 100:(i + 1) * 100]
    for i in range(1000)
]
orig2 = [
    all_points2[i * 100:(i + 1) * 100]
    for i in range(1000)
]
orig1 = np.array(orig1)
orig2 = np.array(orig2)

combined_data1 = np.concatenate((expert_data1, expert_data2), axis=0)
combined_data2 = np.concatenate((orig1, orig2), axis=0)
mean1 = np.mean(combined_data1, axis=(0,1))
std1 = np.std(combined_data1, axis=(0,1))
mean2 = np.mean(combined_data2, axis=(0,1))
std2 = np.std(combined_data2, axis=(0,1))
mean = (mean1 + mean2)/2
std = (std1 + std2)/2
expert_data1 = (expert_data1 - mean) / std
expert_data2 = (expert_data2 - mean) / std
orig1 = (orig1 - mean) / std
orig2 = (orig2 - mean) / std



# Define environment
class TwoUnicycle():
    def __init__(self, state_size=2, action_size=2):
        self.state_size = state_size
        self.action_size = action_size
        self.name = "TwoUnicycle"
env = TwoUnicycle()



# Setting up training data
obs_init1 = expert_data1[:, 0, :]
obs_init2 = expert_data2[:, 0, :]
obs_final1 = np.repeat(orig1[:, -1, :], repeats=10, axis=0)
obs_final2 = np.repeat(orig2[:, -1, :], repeats=10, axis=0)
obs1 = np.hstack([obs_init1, obs_final1])
obs2 = np.hstack([obs_init2, obs_final2])
obs_temp1 = obs1
obs_temp2 = obs2
actions1 = expert_data1[:, :H-1, :]
actions2 = expert_data2[:, :H-1, :]
obs1 = torch.FloatTensor(obs1).to(device)
obs2 = torch.FloatTensor(obs2).to(device)

attr1 = obs1
attr2 = obs2
attr_dim1 = attr1.shape[1]
attr_dim2 = attr2.shape[1]
assert attr_dim1 == env.state_size * 2
assert attr_dim2 == env.state_size * 2

actions1 = torch.FloatTensor(actions1).to(device)
actions2 = torch.FloatTensor(actions2).to(device)
sigma_data1 = actions1.std().item()
sigma_data2 = actions2.std().item()
sig = np.array([sigma_data1, sigma_data2])


print(actions1.shape, actions2.shape, obs1.shape, obs2.shape)

obs = np.hstack([obs1, obs2])
print(obs.shape)

print(attr1)
print(attr2)

torch.Size([10000, 9, 2]) torch.Size([10000, 9, 2]) torch.Size([10000, 4]) torch.Size([10000, 4])
(10000, 8)
tensor([[-1.5334,  0.0288,  1.5117, -0.1075],
        [-1.2252, -0.0133,  1.5117, -0.1075],
        [-0.9194, -0.1458,  1.5117, -0.1075],
        ...,
        [ 0.7165, -0.7212,  1.6371, -0.4495],
        [ 1.1704, -0.4807,  1.6371, -0.4495],
        [ 1.5265, -0.4524,  1.6371, -0.4495]])
tensor([[ 1.5114,  0.0829, -1.5749,  0.0160],
        [ 1.1990,  0.1119, -1.5749,  0.0160],
        [ 0.8894,  0.2365, -1.5749,  0.0160],
        ...,
        [-0.6880, -0.7199, -1.6368, -0.4294],
        [-1.1700, -0.4413, -1.6368, -0.4294],
        [-1.5263, -0.4286, -1.6368, -0.4294]])


In [42]:
# attr_1 = attributes_list[0][idx].clone()  # (batch_size, attr_dim)
# attr_2 = attributes_list[1][idx].clone()  # (batch_size, attr_dim)

# --------------------------------
# STEP 1: Prepare Input for GNN
# --------------------------------

attr1 = attr1[:2, :]
attr2 = attr2[:2, :]

# Stack attr_1 and attr_2 to create a (batch_size, 2, attr_dim) tensor
node_features = torch.stack([attr1, attr2], dim=1)  # Shape: (batch_size, 2, attr_dim)

# Reshape to (batch_size * 2, attr_dim) for GNN input
node_features = node_features.view(-1, attr1.shape[-1])  # Shape: (batch_size * 2, attr_dim)

print(node_features)

batch_size = 2  # Number of trajectories in the batch

edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)  # Shape: (2, 2)

edge_index = edge_index.repeat(1, batch_size)  # Shape: (2, 2 * batch_size)

# Adjust node indices for batched graph (ensuring unique node indices per batch)
offsets = torch.arange(batch_size) * 2  # Offsets for each batch
edge_index = edge_index + offsets.repeat_interleave(2).unsqueeze(0)

print(edge_index)

# --------------------------------
# STEP 2: Pass Through GNN
# --------------------------------
embeddings = gnn(node_features, edge_index)  # Shape: (batch_size * 2, embedding_dim)

print(embeddings)

# Reshape back to (batch_size, 2, embedding_dim)
embeddings = embeddings.view(batch_size, 2, -1)  # Shape: (batch_size, 2, embedding_dim)
# Extract embeddings for each agent
embed_1 = embeddings[:, 0, :]  # Shape: (batch_size, embedding_dim)
embed_2 = embeddings[:, 1, :]  # Shape: (batch_size, embedding_dim)

print(embed_1)
print(embed_2)

gnn(node_features[:2,:],torch.tensor([[0, 1], [1, 0]], dtype=torch.long))

tensor([[-1.5334,  0.0288,  1.5117, -0.1075],
        [ 1.5114,  0.0829, -1.5749,  0.0160],
        [-1.2252, -0.0133,  1.5117, -0.1075],
        [ 1.1990,  0.1119, -1.5749,  0.0160]])
tensor([[0, 1, 2, 3],
        [1, 0, 3, 2]])
tensor([[ 0.6959, -1.2399, -0.2546, -0.1693],
        [ 0.1026,  0.2029, -1.2488,  0.2709],
        [ 0.5578, -1.0204, -0.2243, -0.2247],
        [ 0.1587,  0.1068, -1.1228,  0.2588]], grad_fn=<AddBackward0>)
tensor([[ 0.6959, -1.2399, -0.2546, -0.1693],
        [ 0.5578, -1.0204, -0.2243, -0.2247]], grad_fn=<SliceBackward0>)
tensor([[ 0.1026,  0.2029, -1.2488,  0.2709],
        [ 0.1587,  0.1068, -1.1228,  0.2588]], grad_fn=<SliceBackward0>)


tensor([[ 0.6959, -1.2399, -0.2546, -0.1693],
        [ 0.1026,  0.2029, -1.2488,  0.2709]], grad_fn=<AddBackward0>)

In [12]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

# Define the GNN model
class TwoAgentGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TwoAgentGNN, self).__init__()
        self.conv1 = GraphConv(input_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x  # Output separate embeddings for each agent

# Example input dimensions
input_dim = 4
hidden_dim = 16
output_dim = 4

# Instantiate the GNN
gnn = TwoAgentGNN(input_dim, hidden_dim, output_dim)

obs1_sample = torch.randn(1,4)
obs2_sample = torch.randn(1,4)
x_batch = torch.cat([obs1_sample, obs2_sample], dim=0)

print(x_batch)

# # Example: obs1 and obs2 with shape (10000, 4)
# obs1 = torch.randn(10000, 4)  # Replace with actual data
# obs2 = torch.randn(10000, 4)  # Replace with actual data

# # Stack observations to form a batch of agent pairs
# x_batch = torch.cat([obs1, obs2], dim=0)  # Shape: (20000, 4)

# # Define edge connections (same for all batches)
edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)  # Shape: (2, 2)

# # Expand edge_index for batch processing
# batch_size = obs1.shape[0]  # 10000
# edge_index = edge_index.repeat(1, batch_size)  # Shape: (2, 2 * batch_size)

# # Adjust node indices for batched graph (ensuring unique node indices per batch)
# offsets = torch.arange(batch_size) * 2  # Offsets for each batch
# edge_index = edge_index + offsets.repeat_interleave(2).unsqueeze(0)

# # Forward pass through GNN
# embeddings = gnn(x_batch, edge_index)  # Shape: (20000, 4)

# # Reshape to separate agents
# embeddings1, embeddings2 = embeddings.chunk(2, dim=0)  # Both shapes: (10000, 4)

# print("Embeddings for Agent 1:", embeddings1.shape)  # Expected: (10000, 4)
# print("Embeddings for Agent 2:", embeddings2.shape)  # Expected: (10000, 4)

tensor([[-0.0591, -1.0592,  0.8310,  0.5888],
        [ 0.7026,  0.6647, -0.5864, -0.5164]])


In [14]:
gnn(x_batch, edge_index)

tensor([[ 0.2865, -0.6232, -0.4839, -0.4187],
        [ 0.5003,  0.0306, -0.9184,  0.1090]], grad_fn=<AddBackward0>)

In [ ]:
em